In [136]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [137]:
import json
import os, sys
import numpy as np
import pandas as pd

current_path = os.getcwd()
sys.path.append(os.path.join(current_path, '..'))

from adapt_model import parse_experiment_name
from utils.globals import *


In [138]:
!ls $RESULTS_DIR

coref_res.csv	       gen_res_neutral.csv  original
coref_res_neutral.csv  gen_res_nw_7B.csv    results_known_disable_mlp_10.jsonl
coref_res_nw_7B.csv    lm_res.csv	    ROME
DAMA		       lm_res_neutral.csv
gen_res.csv	       lm_res_nw_7B.csv


In [139]:
model_name = 'llama_13B'

default_params = {
    'iterative_update': False,
    'mixed_update': True,
    'task': 'gen',
    'post_linear': True,
    'batch_size': 1,
    'orthogonal_constraint': 0.0,
    'no_colinear_vs': True,
    'use_neutral': True,
    'vs_at_last': True,
    'delta_only': True,
    'nw': True
}

variable_parameters = {
    'num_layers': [8, 10, 12,14],
    'null_dim': [32,64, 128, 256, 512, 1024]
}



In [140]:
def load_data_file(model_name, method, experiment_name, test_file):
    if experiment_name and 'method' != 'original':
        result_dir = os.path.join(model_name,experiment_name)
    else:
        result_dir = model_name
    result_file = os.path.join(RESULTS_DIR, method, result_dir, test_file)
    try:
        with open(result_file, 'r') as f:
            results = json.load(f)
    except FileNotFoundError:
        print(f"File {result_file} not found")
        results = None

    return results

Process generation results

In [141]:
def parse_generation_results(layers, dimensions, results):
    out_row = {'layers': layers, 'dimensions': dimensions}
    for key in (
            #'slope_s', 'intercept_s', 'slope_f',
            'intercept_f', 'joint_slope_s', 'joint_slope_f', 'joint_intercept', 'prob_he', 'prob_she', 'prob_they', 'spearman_s', 'spearman_f'):
        if results is None:
            out_row[key] = 0.
        else:
            out_row[key] = results[key]
    return out_row

In [142]:
gen_res = pd.DataFrame(columns=['layers', 'dimensions',
                                #'slope_s', 'intercept_s', 'slope_f', 'intercept_f',
                                'joint_slope_s', 'joint_slope_f', 'joint_intercept','prob_he', 'prob_she', 'prob_they',
                                'spearman_s', 'spearman_f'])
test_file = 'res_gen_test_dama.json'
# row = {'layers': None,'dimensions': None, 'slope_s': 0. ,'intercept_s': 0., 'slope_f': 0., 'intercept_f': 0.,
#        'prob_he': 0., 'prob_she': 0., 'prob_they': 0.}

tmp_res = load_data_file(model_name, 'original', None, test_file)

gen_res.loc[len(gen_res)] = parse_generation_results(0, 0, tmp_res)

for num_layers in variable_parameters['num_layers']:
    for null_dim in variable_parameters['null_dim']:
        vs_at_last = True
        experiment_name = parse_experiment_name(**default_params, num_layers=num_layers, null_dim=null_dim)
        tmp_res = load_data_file(model_name, 'DAMA', experiment_name, test_file)
        gen_res.loc[len(gen_res)] = parse_generation_results(num_layers, null_dim, tmp_res)

File ../../../results/DAMA/llama_13B/l8_on_nd32_ncv_val_neutral_delta_only_nw/res_gen_test_dama.json not found
File ../../../results/DAMA/llama_13B/l8_on_nd256_ncv_val_neutral_delta_only_nw/res_gen_test_dama.json not found
File ../../../results/DAMA/llama_13B/l10_on_nd32_ncv_val_neutral_delta_only_nw/res_gen_test_dama.json not found
File ../../../results/DAMA/llama_13B/l10_on_nd256_ncv_val_neutral_delta_only_nw/res_gen_test_dama.json not found
File ../../../results/DAMA/llama_13B/l12_on_nd32_ncv_val_neutral_delta_only_nw/res_gen_test_dama.json not found
File ../../../results/DAMA/llama_13B/l12_on_nd256_ncv_val_neutral_delta_only_nw/res_gen_test_dama.json not found
File ../../../results/DAMA/llama_13B/l14_on_nd32_ncv_val_neutral_delta_only_nw/res_gen_test_dama.json not found
File ../../../results/DAMA/llama_13B/l14_on_nd256_ncv_val_neutral_delta_only_nw/res_gen_test_dama.json not found


In [143]:
display(gen_res)
# save to csv
gen_res.to_csv(os.path.join(RESULTS_DIR, 'gen_res.csv'), index=False)

,layers,dimensions,joint_slope_s,joint_slope_f,joint_intercept,prob_he,prob_she,prob_they,spearman_s,spearman_f
0,0,0,0.270104,0.350547,0.067949,0.284406,0.088571,0.021878,0.428092,0.592075
1,8,32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,8,64,0.217992,0.306501,0.047397,0.245614,0.090755,0.030677,0.429735,0.623577
3,8,128,0.212114,0.301825,0.048876,0.244836,0.090748,0.030086,0.429533,0.623456
4,8,256,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,8,512,0.187360,0.280433,0.047369,0.234479,0.091684,0.030426,0.424140,0.622560
6,8,1024,0.174635,0.266822,0.049124,0.228105,0.089057,0.030887,0.419574,0.616135
7,10,32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,10,64,0.201679,0.278888,0.058910,0.232310,0.074821,0.035420,0.422702,0.595422
9,10,128,0.187992,0.265056,0.059988,0.226544,0.073751,0.033320,0.418238,0.586191


Process causal lm results

In [144]:
def parse_causal_lm_results(layers, dimensions, results):
    out_row = {'layers': layers, 'dimensions': dimensions, 'perplexity': results['mean_perplexity'] if results else 0.}
    return out_row

In [145]:
lm_res = pd.DataFrame(columns=['layers', 'dimensions', 'perplexity'])

test_file = 'res_causal_lm_wikitext_wikitext-103-raw-v1.json'
# row = {'layers': None,'dimensions': None, 'perplexity': 0.}

tmp_res = load_data_file(model_name, 'original', None, test_file)
lm_res.loc[len(lm_res)] = parse_causal_lm_results(0, 0, tmp_res)

for num_layers in variable_parameters['num_layers']:
    for null_dim in variable_parameters['null_dim']:
        vs_at_last = True
        experiment_name = parse_experiment_name(**default_params, num_layers=num_layers, null_dim=null_dim)
        tmp_res = load_data_file(model_name, 'DAMA', experiment_name, test_file)
        lm_res.loc[len(lm_res)] = parse_causal_lm_results(num_layers, null_dim, tmp_res)

File ../../../results/DAMA/llama_13B/l8_on_nd32_ncv_val_neutral_delta_only_nw/res_causal_lm_wikitext_wikitext-103-raw-v1.json not found
File ../../../results/DAMA/llama_13B/l8_on_nd256_ncv_val_neutral_delta_only_nw/res_causal_lm_wikitext_wikitext-103-raw-v1.json not found
File ../../../results/DAMA/llama_13B/l10_on_nd32_ncv_val_neutral_delta_only_nw/res_causal_lm_wikitext_wikitext-103-raw-v1.json not found
File ../../../results/DAMA/llama_13B/l10_on_nd256_ncv_val_neutral_delta_only_nw/res_causal_lm_wikitext_wikitext-103-raw-v1.json not found
File ../../../results/DAMA/llama_13B/l12_on_nd32_ncv_val_neutral_delta_only_nw/res_causal_lm_wikitext_wikitext-103-raw-v1.json not found
File ../../../results/DAMA/llama_13B/l12_on_nd256_ncv_val_neutral_delta_only_nw/res_causal_lm_wikitext_wikitext-103-raw-v1.json not found
File ../../../results/DAMA/llama_13B/l14_on_nd32_ncv_val_neutral_delta_only_nw/res_causal_lm_wikitext_wikitext-103-raw-v1.json not found
File ../../../results/DAMA/llama_13B/l14

In [146]:
display(lm_res)
# save to csv
lm_res.to_csv(os.path.join(RESULTS_DIR, 'lm_res.csv'), index=False)

,layers,dimensions,perplexity
0,0,0,19.827272
1,8,32,0.000000
2,8,64,19.881645
3,8,128,19.656230
4,8,256,0.000000
5,8,512,18.972118
6,8,1024,19.834539
7,10,32,0.000000
8,10,64,17.759147
9,10,128,18.925584


Process coreference results

In [147]:
def parse_coref_results(layers, dimensions, results_a1, results_a2, results_p1, results_p2):
    out_row = {'layers': layers, 'dimensions': dimensions}

    if results_a1 is None or results_a2 is None or results_p1 is None or results_p2 is None:
        out_row['anti_male'] = 0.
        out_row['anti_female'] = 0.
        out_row['pro_male'] = 0.
        out_row['pro_female'] = 0.
    else:
        out_row['anti_male'] = (results_a1['m_acc'] + results_a2['m_acc']) / 2.
        out_row['anti_female'] = (results_a1['f_acc'] + results_a2['f_acc']) / 2.
        out_row['pro_male'] = (results_p1['m_acc'] + results_p2['m_acc']) / 2.
        out_row['pro_female'] = (results_p1['f_acc'] + results_p2['f_acc']) / 2.

    out_row['ACC'] = (out_row['anti_male'] + out_row['anti_female'] + out_row['pro_male'] + out_row['pro_female']) / 4.
    out_row['Delta_S'] = (out_row['pro_male'] + out_row['pro_female'] - out_row['anti_male'] - out_row['anti_female']) / 2.
    out_row['Delta_G'] = (out_row['pro_male'] + out_row['anti_male'] - out_row['pro_female'] - out_row['anti_female']) / 2.
    return out_row

In [148]:
lm_rs = pd.DataFrame(columns=['layers', 'dimensions', 'ACC', 'Delta_S', 'Delta_G', 'anti_male', 'anti_female', 'pro_male', 'pro_female'])

test_file_a1 = 'res_coref_anti_type1_test.json'
test_file_a2 = 'res_coref_anti_type2_test.json'
test_file_p1 = 'res_coref_pro_type1_test.json'
test_file_p2 = 'res_coref_pro_type2_test.json'


tmp_res_a1 = load_data_file(model_name, 'original', None, test_file_a1)
tmp_res_a2 = load_data_file(model_name, 'original', None, test_file_a2)
tmp_res_p1 = load_data_file(model_name, 'original', None, test_file_p1)
tmp_res_p2 = load_data_file(model_name, 'original', None, test_file_p2)

lm_rs.loc[len(lm_rs)] = parse_coref_results(0, 0, tmp_res_a1, tmp_res_a2, tmp_res_p1, tmp_res_p2)

for num_layers in variable_parameters['num_layers']:
    for null_dim in variable_parameters['null_dim']:
        vs_at_last = True
        experiment_name = parse_experiment_name(**default_params, num_layers=num_layers, null_dim=null_dim)
        tmp_res_a1 = load_data_file(model_name, 'DAMA', experiment_name, test_file_a1)
        tmp_res_a2 = load_data_file(model_name, 'DAMA', experiment_name, test_file_a2)
        tmp_res_p1 = load_data_file(model_name, 'DAMA', experiment_name, test_file_p1)
        tmp_res_p2 = load_data_file(model_name, 'DAMA', experiment_name, test_file_p2)
        lm_rs.loc[len(lm_rs)] = parse_coref_results(num_layers, null_dim, tmp_res_a1, tmp_res_a2, tmp_res_p1, tmp_res_p2)



File ../../../results/DAMA/llama_13B/l8_on_nd32_ncv_val_neutral_delta_only_nw/res_coref_anti_type1_test.json not found
File ../../../results/DAMA/llama_13B/l8_on_nd32_ncv_val_neutral_delta_only_nw/res_coref_anti_type2_test.json not found
File ../../../results/DAMA/llama_13B/l8_on_nd32_ncv_val_neutral_delta_only_nw/res_coref_pro_type1_test.json not found
File ../../../results/DAMA/llama_13B/l8_on_nd32_ncv_val_neutral_delta_only_nw/res_coref_pro_type2_test.json not found
File ../../../results/DAMA/llama_13B/l8_on_nd256_ncv_val_neutral_delta_only_nw/res_coref_anti_type1_test.json not found
File ../../../results/DAMA/llama_13B/l8_on_nd256_ncv_val_neutral_delta_only_nw/res_coref_anti_type2_test.json not found
File ../../../results/DAMA/llama_13B/l8_on_nd256_ncv_val_neutral_delta_only_nw/res_coref_pro_type1_test.json not found
File ../../../results/DAMA/llama_13B/l8_on_nd256_ncv_val_neutral_delta_only_nw/res_coref_pro_type2_test.json not found
File ../../../results/DAMA/llama_13B/l10_on_nd32

In [149]:
display(lm_rs)
# save to csv
lm_rs.to_csv(os.path.join(RESULTS_DIR, 'coref_res.csv'), index=False)

,layers,dimensions,ACC,Delta_S,Delta_G,anti_male,anti_female,pro_male,pro_female
0,0,0,0.705200,0.357277,-0.014718,0.496692,0.556432,0.898990,0.868687
1,8,32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,8,64,0.703274,0.345977,-0.003367,0.516818,0.543754,0.886364,0.866162
3,8,128,0.696285,0.324601,-0.004667,0.531843,0.536127,0.856061,0.861111
4,8,256,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,8,512,0.667289,0.296736,0.003125,0.511944,0.525897,0.825758,0.805556
6,8,1024,0.640174,0.255006,-0.008176,0.499444,0.525897,0.772727,0.762626
7,10,32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,10,64,0.698861,0.342176,-0.004617,0.514318,0.541229,0.878788,0.861111
9,10,128,0.695080,0.321961,-0.009680,0.521894,0.546305,0.858586,0.853535
